In [1]:
from bs4 import BeautifulSoup
from string import ascii_uppercase
import requests
import re
import pandas as pd

In [2]:
initial_link = 'https://basketball.realgm.com'

In [3]:
draft_link = 'https://www.basketball-reference.com/draft'

In [5]:
all_players = []

In [6]:
keys = ['Season','Name','Age','Team','Guard', 'Forward', 'Center', 'Big_conference', 'Weight', 'Height', 'GP', 'GS', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OFF', 'DEF', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'Drafted']

In [7]:
big_confs = ['Big Ten', 'Southwestern Athletic', 'Big East', 'Southeastern', 'Metro Atlantic', 'Atlantic Coast', 'Pacific 12', 'Big 12']

In [9]:
for year in range (2017,2022):
    
   season = str(int(year) - 1) + '-' + str(year)[2:4]

   root_draft = f'{draft_link}/NBA_{year}.html'
   request_draft = requests.get(root_draft)
   soup_draft = BeautifulSoup(request_draft.content, 'lxml')
   draft_stat = str(soup_draft.find(name='table'))

    
   for letter in (ascii_uppercase):
      root = f'{initial_link}/ncaa/players/{year}/{letter}'
      request = requests.get(root)
      soup = BeautifulSoup(request.content, 'lxml')
      sum_stat = soup.find(name='table')
    
      for row in sum_stat.find_all('tr')[1:]:
         values = []
         values.append(year)
         values.append(row.find('a').text.strip())
         name = values[1]
                
         sum = str(row.find('a'))
         href = sum.split('"')[1]
         link = (f'https://basketball.realgm.com{href}')
         req_new = requests.get(link)
         soup_new = BeautifulSoup(req_new.content, 'lxml')
        
         s = str(soup_new.find_all('p'))
         s_n_str = str(soup_new)
        
         a = str(re.search('"(.*)years old', s_n_str))
         a_1 = a.replace(' ','')
         
         if a_1 != 'None':
            age = int(a_1[-3:-1])
         else:
            age = ''
         values.append(age)      
        
         v = str(soup_new.find_all('head'))
         x = str(re.findall('<title>(.*)</title>', v))
         t_1 = x.split(' ')
            
         if t_1[3] == 'Profile,':
            t_2 = t_1[4:7]
         elif t_1[3] == 'Player':
            t_2 = t_1[5:8]
         else:
            pass
            
        
         if t_2[2] == 'NCAA':
            t_3 = ' '.join(t_2[0:2])
         elif t_2[1] == 'NCAA':
            t_3 = t_2[0]
         elif t_2[2] == 'News,':
            t_3 = ' '.join(t_2[0:2])
         else:
            t_3 = ' '.join(t_2)
            
         team = t_3.replace(',','')
         values.append(team)
            

         p = str(re.search('<span class="feature">(.*)</span>', s_n_str))
         p_1 = ''.join(p.split()[5])
         p_2 = list(p_1)
        
         if p_2[17]=='<':
                position = p_1[16]
         else:
                position = ''.join(p_1[16:18])
                
         if position == 'PG' or position == 'SG' or position == 'G':
            position = 1
         elif position == 'SF' or position == 'PF' or position == 'F':
            position = 2
         else:
            position = 3
        
        
         if position == 1:
            guard = 1
         else:
            guard = 0
         values.append(guard)
        
         if position == 2:
            forward = 1
         else:
            forward = 0
         values.append(forward)
            
         if position == 3:
            center = 1
         else:
            center = 0
         values.append(center)
        
        
         t = str(soup_new.find_all('a'))
         conf_ = str(re.search('href="/ncaa/conferences(.*)</a>', t))
         conf_1 = ''.join(conf_.split()[4])
         conf_2 = conf_1[30:]
         conf_3 = conf_2.partition('/')[2]
         conf_4 = conf_3.partition('/')[0]
         conf_5 = conf_4.replace('-',' ')
         conf_6 = ' '.join(conf_5.split(' ')[:2])
         conf_7 = conf_6.split(' ')[1]
         if conf_7 == 'Conference':
            conf = ''.join(conf_6.split(' ')[0])
         else:
            conf = conf_6
        
         if conf in big_confs:
            big_conf = 1
         else:
            big_conf = 0
         values.append(big_conf)
            

         w = str(re.search('Weight:</strong> (.*)</p>', s))
         w_1 = w.split(' ')[6]
         weight = "".join(c for c in w_1 if  c.isdecimal())
         values.append(weight)
        
         h = str(re.search('>Height:</strong> (.*)', s))
         h_1 = h.split(' ')[6]
         height = float("".join(c for c in h_1 if  c.isdecimal()))
         values.append(height)
        
         stat_new = str(soup_new.find_all(name = 'tbody'))
         stat_new = re.sub('<td id=(.*)</td>','',stat_new)
        
         stat_new = stat_new.replace('td','')
         stat_new = stat_new.replace('tr','')
         stat_new = stat_new.replace('/','')
         stat_new = stat_new.replace('<>','')
         stat_new = stat_new.replace(' ','')
         stat_new = stat_new.replace('<class="per_game">','')        
            
         sheeesh = list(stat_new.split())
         ind = sheeesh.index(season)  
         for z in range (ind+1, ind +22):
            if sheeesh[z]!= '-':
                values.append(sheeesh[z])
            else:
                values.append('0')
                

         drafted =re.search(name, draft_stat)
                        
         if drafted is None:
            drafted = 0
         else:  
            drafted = 1
            
         values.append(drafted)
            
         player = dict(zip(keys,values))
         all_players.append(player)
         print(player)


{'Season': 2020, 'Name': 'Eli Abaev', 'Age': 24, 'Team': 'Florida Gulf Coast', 'Guard': 0, 'Forward': 1, 'Center': 0, 'Big_conference': 0, 'Weight': '95', 'Height': 203.0, 'GP': '30', 'GS': '30', 'MIN': '28.0', 'PTS': '7.93', 'FGM': '2.93', 'FGA': '5.80', 'FG%': '.506', '3PM': '0.00', '3PA': '0.07', '3P%': '.000', 'FTM': '2.07', 'FTA': '3.67', 'FT%': '.564', 'OFF': '2.57', 'DEF': '5.17', 'TRB': '7.73', 'AST': '1.13', 'STL': '0.87', 'BLK': '0.70', 'TOV': '1.53', 'PF': '3.00', 'Drafted': 0}
{'Season': 2020, 'Name': 'Mohamed Abdusalam', 'Age': 24, 'Team': 'UNC Greensboro', 'Guard': 0, 'Forward': 0, 'Center': 1, 'Big_conference': 0, 'Weight': '116', 'Height': 206.0, 'GP': '31', 'GS': '1', 'MIN': '16.4', 'PTS': '3.77', 'FGM': '1.71', 'FGA': '3.39', 'FG%': '.505', '3PM': '0.00', '3PA': '0.00', '3P%': '.000', 'FTM': '0.35', 'FTA': '0.97', 'FT%': '.367', 'OFF': '1.29', 'DEF': '2.16', 'TRB': '3.45', 'AST': '0.61', 'STL': '0.71', 'BLK': '0.48', 'TOV': '0.77', 'PF': '2.45', 'Drafted': 0}
{'Season

{'Season': 2020, 'Name': 'Darian Adams', 'Age': 24, 'Team': 'Jacksonville State', 'Guard': 1, 'Forward': 0, 'Center': 0, 'Big_conference': 0, 'Weight': '91', 'Height': 191.0, 'GP': '25', 'GS': '21', 'MIN': '31.3', 'PTS': '12.12', 'FGM': '3.96', 'FGA': '11.52', 'FG%': '.344', '3PM': '2.28', '3PA': '6.84', '3P%': '.333', 'FTM': '1.92', 'FTA': '2.32', 'FT%': '.828', 'OFF': '1.16', 'DEF': '3.28', 'TRB': '4.44', 'AST': '3.28', 'STL': '0.88', 'BLK': '0.20', 'TOV': '2.64', 'PF': '2.48', 'Drafted': 0}
{'Season': 2020, 'Name': 'Jordyn Adams', 'Age': 21, 'Team': 'Lamar', 'Guard': 1, 'Forward': 0, 'Center': 0, 'Big_conference': 0, 'Weight': '91', 'Height': 191.0, 'GP': '33', 'GS': '31', 'MIN': '33.1', 'PTS': '17.39', 'FGM': '5.85', 'FGA': '13.94', 'FG%': '.420', '3PM': '1.82', '3PA': '4.82', '3P%': '.377', 'FTM': '3.88', 'FTA': '4.91', 'FT%': '.790', 'OFF': '0.82', 'DEF': '2.42', 'TRB': '3.24', 'AST': '2.09', 'STL': '0.94', 'BLK': '0.27', 'TOV': '2.42', 'PF': '2.06', 'Drafted': 0}
{'Season': 2020

{'Season': 2020, 'Name': 'Arnas Adomavicius', 'Age': 22, 'Team': 'UNC Greensboro', 'Guard': 1, 'Forward': 0, 'Center': 0, 'Big_conference': 0, 'Weight': '91', 'Height': 193.0, 'GP': '0', 'GS': '0', 'MIN': '0', 'PTS': '0', 'FGM': '0', 'FGA': '0', 'FG%': '0', '3PM': '0', '3PA': '0', '3P%': '0', 'FTM': '0', 'FTA': '0', 'FT%': '0', 'OFF': '0', 'DEF': '0', 'TRB': '0', 'AST': '0', 'STL': '0', 'BLK': '0', 'TOV': '0', 'PF': '0', 'Drafted': 0}
{'Season': 2020, 'Name': 'Aguir Agau', 'Age': '', 'Team': 'Fresno State', 'Guard': 1, 'Forward': 0, 'Center': 0, 'Big_conference': 0, 'Weight': '86', 'Height': 201.0, 'GP': '23', 'GS': '10', 'MIN': '16.5', 'PTS': '4.17', 'FGM': '1.39', 'FGA': '3.30', 'FG%': '.421', '3PM': '0.52', '3PA': '1.74', '3P%': '.300', 'FTM': '0.87', 'FTA': '1.17', 'FT%': '.741', 'OFF': '0.78', 'DEF': '2.26', 'TRB': '3.04', 'AST': '0.52', 'STL': '0.35', 'BLK': '0.48', 'TOV': '1.22', 'PF': '2.35', 'Drafted': 0}
{'Season': 2020, 'Name': 'Ochai Agbaji', 'Age': 21, 'Team': 'Kansas', 'G

{'Season': 2020, 'Name': 'Jamie Ahale', 'Age': 23, 'Team': 'Illinois-Chicago', 'Guard': 0, 'Forward': 0, 'Center': 1, 'Big_conference': 0, 'Weight': '93', 'Height': 196.0, 'GP': '35', 'GS': '7', 'MIN': '20.3', 'PTS': '5.26', 'FGM': '1.54', 'FGA': '4.17', 'FG%': '.370', '3PM': '1.31', '3PA': '3.83', '3P%': '.343', 'FTM': '0.86', 'FTA': '1.06', 'FT%': '.811', 'OFF': '0.54', 'DEF': '2.11', 'TRB': '2.66', 'AST': '0.86', 'STL': '0.46', 'BLK': '0.31', 'TOV': '0.91', 'PF': '1.46', 'Drafted': 0}
{'Season': 2020, 'Name': 'Akrum Ahemed', 'Age': 21, 'Team': 'Canisius', 'Guard': 0, 'Forward': 1, 'Center': 0, 'Big_conference': 1, 'Weight': '91', 'Height': 196.0, 'GP': '9', 'GS': '6', 'MIN': '9.9', 'PTS': '3.00', 'FGM': '1.11', 'FGA': '2.33', 'FG%': '.476', '3PM': '0.33', '3PA': '0.78', '3P%': '.429', 'FTM': '0.44', 'FTA': '0.56', 'FT%': '.800', 'OFF': '0.33', 'DEF': '0.56', 'TRB': '0.89', 'AST': '0.78', 'STL': '0.33', 'BLK': '0.00', 'TOV': '1.56', 'PF': '1.11', 'Drafted': 0}
{'Season': 2020, 'Name'

{'Season': 2020, 'Name': 'Ayo Akinwole', 'Age': 23, 'Team': 'Drake', 'Guard': 1, 'Forward': 0, 'Center': 0, 'Big_conference': 0, 'Weight': '82', 'Height': 180.0, 'GP': '32', 'GS': '21', 'MIN': '27.3', 'PTS': '7.03', 'FGM': '2.72', 'FGA': '5.88', 'FG%': '.463', '3PM': '0.78', '3PA': '2.19', '3P%': '.357', 'FTM': '0.81', 'FTA': '1.12', 'FT%': '.722', 'OFF': '0.38', 'DEF': '3.38', 'TRB': '3.75', 'AST': '2.66', 'STL': '0.84', 'BLK': '0.12', 'TOV': '1.25', 'PF': '1.69', 'Drafted': 0}
{'Season': 2020, 'Name': 'Akok Akok', 'Age': 22, 'Team': 'Connecticut', 'Guard': 0, 'Forward': 1, 'Center': 0, 'Big_conference': 1, 'Weight': '95', 'Height': 206.0, 'GP': '25', 'GS': '24', 'MIN': '28.0', 'PTS': '5.80', 'FGM': '2.28', 'FGA': '5.52', 'FG%': '.413', '3PM': '0.48', '3PA': '1.84', '3P%': '.261', 'FTM': '0.76', 'FTA': '1.28', 'FT%': '.594', 'OFF': '1.32', 'DEF': '4.20', 'TRB': '5.52', 'AST': '0.44', 'STL': '0.36', 'BLK': '2.64', 'TOV': '1.00', 'PF': '1.60', 'Drafted': 0}
{'Season': 2020, 'Name': 'Jon

...

{'Season': 2021, 'Name': 'Mladen Armus', 'Age': 24, 'Team': 'Boise State', 'Guard': 0, 'Forward': 1, 'Center': 0, 'Big_conference': 0, 'Weight': '105', 'Height': 208.0, 'GP': '28', 'GS': '28', 'MIN': '26.7', 'PTS': '8.11', 'FGM': '2.82', 'FGA': '5.32', 'FG%': '.530', '3PM': '0.00', '3PA': '0.04', '3P%': '.000', 'FTM': '2.46', 'FTA': '3.57', 'FT%': '.690', 'OFF': '3.18', 'DEF': '4.64', 'TRB': '7.82', 'AST': '1.18', 'STL': '0.54', 'BLK': '0.71', 'TOV': '1.43', 'PF': '2.96', 'Drafted': 0}
{'Season': 2021, 'Name': 'Dylan Arnette', 'Age': 22, 'Team': 'Western New Mexico', 'Guard': 1, 'Forward': 0, 'Center': 0, 'Big_conference': 1, 'Weight': '79', 'Height': 191.0, 'GP': '1', 'GS': '0', 'MIN': '0.0', 'PTS': '0.00', 'FGM': '0.00', 'FGA': '0.00', 'FG%': '.000', '3PM': '0.00', '3PA': '0.00', '3P%': '.000', 'FTM': '0.00', 'FTA': '0.00', 'FT%': '.000', 'OFF': '0.00', 'DEF': '0.00', 'TRB': '0.00', 'AST': '0.00', 'STL': '0.00', 'BLK': '0.00', 'TOV': '0.00', 'PF': '0.00', 'Drafted': 0}
{'Season': 202

{'Season': 2021, 'Name': 'Carter Ash', 'Age': '', 'Team': 'Montana State', 'Guard': 1, 'Forward': 0, 'Center': 0, 'Big_conference': 0, 'Weight': '75', 'Height': 183.0, 'GP': '0', 'GS': '0', 'MIN': '0', 'PTS': '0', 'FGM': '0', 'FGA': '0', 'FG%': '0', '3PM': '0', '3PA': '0', '3P%': '0', 'FTM': '0', 'FTA': '0', 'FT%': '0', 'OFF': '0', 'DEF': '0', 'TRB': '0', 'AST': '0', 'STL': '0', 'BLK': '0', 'TOV': '0', 'PF': '0', 'Drafted': 0}
{'Season': 2021, 'Name': 'Makai Ashton-Langford', 'Age': 24, 'Team': 'Boston College', 'Guard': 1, 'Forward': 0, 'Center': 0, 'Big_conference': 1, 'Weight': '84', 'Height': 191.0, 'GP': '15', 'GS': '12', 'MIN': '26.4', 'PTS': '9.67', 'FGM': '3.40', 'FGA': '8.13', 'FG%': '.418', '3PM': '0.80', '3PA': '3.40', '3P%': '.235', 'FTM': '2.07', 'FTA': '3.13', 'FT%': '.660', 'OFF': '0.47', 'DEF': '2.87', 'TRB': '3.33', 'AST': '3.20', 'STL': '1.73', 'BLK': '0.07', 'TOV': '1.67', 'PF': '3.33', 'Drafted': 0}
{'Season': 2021, 'Name': 'Steven Ashworth', 'Age': '', 'Team': 'Uta

{'Season': 2021, 'Name': 'Eric Ayala', 'Age': 23, 'Team': 'Maryland', 'Guard': 1, 'Forward': 0, 'Center': 0, 'Big_conference': 1, 'Weight': '91', 'Height': 196.0, 'GP': '29', 'GS': '28', 'MIN': '33.5', 'PTS': '15.10', 'FGM': '4.79', 'FGA': '10.97', 'FG%': '.437', '3PM': '1.97', '3PA': '5.83', '3P%': '.337', 'FTM': '3.55', 'FTA': '4.28', 'FT%': '.831', 'OFF': '0.52', 'DEF': '3.76', 'TRB': '4.28', 'AST': '2.17', 'STL': '1.17', 'BLK': '0.14', 'TOV': '1.66', 'PF': '1.38', 'Drafted': 0}
{'Season': 2021, 'Name': 'Stephane Ayangma', 'Age': '', 'Team': 'Central Connecticut State', 'Guard': 0, 'Forward': 1, 'Center': 0, 'Big_conference': 0, 'Weight': '98', 'Height': 198.0, 'GP': '21', 'GS': '12', 'MIN': '17.8', 'PTS': '6.29', 'FGM': '2.52', 'FGA': '5.14', 'FG%': '.491', '3PM': '0.43', '3PA': '1.52', '3P%': '.281', 'FTM': '0.81', 'FTA': '1.19', 'FT%': '.680', 'OFF': '1.33', 'DEF': '3.29', 'TRB': '4.62', 'AST': '0.57', 'STL': '0.52', 'BLK': '0.29', 'TOV': '1.38', 'PF': '1.29', 'Drafted': 0}
{'Sea

{'Season': 2021, 'Name': 'Armando Bacot', 'Age': 22, 'Team': 'North Carolina', 'Guard': 0, 'Forward': 1, 'Center': 0, 'Big_conference': 1, 'Weight': '109', 'Height': 208.0, 'GP': '29', 'GS': '28', 'MIN': '22.7', 'PTS': '12.31', 'FGM': '4.72', 'FGA': '7.52', 'FG%': '.628', '3PM': '0.00', '3PA': '0.03', '3P%': '.000', 'FTM': '2.86', 'FTA': '4.31', 'FT%': '.664', 'OFF': '3.17', 'DEF': '4.66', 'TRB': '7.83', 'AST': '0.79', 'STL': '0.66', 'BLK': '0.90', 'TOV': '1.41', 'PF': '2.24', 'Drafted': 0}
{'Season': 2021, 'Name': 'Cam Bacote', 'Age': '', 'Team': 'Western Carolina', 'Guard': 1, 'Forward': 0, 'Center': 0, 'Big_conference': 0, 'Weight': '78', 'Height': 188.0, 'GP': '17', 'GS': '0', 'MIN': '11.4', 'PTS': '2.71', 'FGM': '1.06', 'FGA': '2.18', 'FG%': '.486', '3PM': '0.53', '3PA': '1.06', '3P%': '.500', 'FTM': '0.06', 'FTA': '0.12', 'FT%': '.500', 'OFF': '0.12', 'DEF': '0.65', 'TRB': '0.76', 'AST': '0.65', 'STL': '0.12', 'BLK': '0.00', 'TOV': '0.41', 'PF': '0.47', 'Drafted': 0}
{'Season': 2

KeyboardInterrupt: 

In [10]:
pd_all_pl = pd.DataFrame(all_players)
print(pd_all_pl)

writer = pd.ExcelWriter('dataset.xlsx')
pd_all_pl.to_excel(writer)

writer.save()

      Season                       Name Age                Team  Guard  \
0       2020                  Eli Abaev  24  Florida Gulf Coast      0   
1       2020          Mohamed Abdusalam  24      UNC Greensboro      0   
2       2020  Agah Abdussamed Kizilkaya  22               Maine      1   
3       2020              Fletcher Abee                 Citadel      1   
4       2020          Riley Abercrombie                    Rice      0   
...      ...                        ...  ..                 ...    ...   
8325    2021            Jeremiah Bailey                 Pacific      0   
8326    2021         Victor Bailey, Jr.  23           Tennessee      1   
8327    2021             Malcolm Bailey  21             Colgate      0   
8328    2021               Charles Bain  25       Robert Morris      0   
8329    2021            Evan Bainbridge                 Ashland      0   

      Forward  Center  Big_conference Weight  Height  ...   FT%   OFF   DEF  \
0           1       0           